In [78]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [79]:
theater_seat_organizer = gp.Model()

# The Parameters

In [80]:
number_of_customers_in_group = [3, 5, 6] 

customer_vaccination_status = [
    [0,0,0], 
    [1,1,1,1,1],
    [0,1,0,0,1,1]
]

number_of_seats = 40

non_vax_cost = 5

# Parameter Validation Logic

In [81]:
def isBinary(num):
    if(num == 1 or num == 0):
        return True
    else:
        return False


# Vaccine Records Should Match Customer Records
number_of_vax = count = sum( [ len(customers) for customers in customer_vaccination_status])

if(sum(number_of_customers_in_group) != number_of_vax):
    raise ValueError('Vaccine Records Do Not Match Customer Records')
    
# Vaccine Records Should be Formatted like Customer Records
for index, customer_ofGroup_vaxList in enumerate(customer_vaccination_status):
    if any(not isBinary(person) for person in customer_ofGroup_vaxList):
        raise ValueError('Invalid Arg: Person is either vaxed = 1 or unvaxxed = 0')
        
    if(len(customer_ofGroup_vaxList) != number_of_customers_in_group[index]):
        raise ValueError('Vaccine Records Are Not Formatted Correctly')



# Decision Variables

In [82]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [83]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= non_vax_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [84]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [85]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")


In [86]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [92]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

In [93]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 4971 rows, 563 columns and 11511 nonzeros
Model fingerprint: 0x48d1a89f
Variable types: 0 continuous, 563 integer (563 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint person2_inGroup2_andPerson4+inGroup1_cannotSitTogether by 1.000000000

Found heuristic solution: objective -15.0000000
Presolve removed 4488 rows and 0 columns
Presolve time: 0.03s
Presolved: 483 rows, 563 columns, 6573 nonzeros
Variable types: 0 continuous, 563 integer (563 binary)

Root relaxation: objective 1.400000e+01, 188 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl U

In [94]:
theater_seat_organizer.getConstrs()

[<gurobi.Constr at_most_one_person_at_seat0>,
 <gurobi.Constr at_most_one_person_at_seat1>,
 <gurobi.Constr at_most_one_person_at_seat2>,
 <gurobi.Constr at_most_one_person_at_seat3>,
 <gurobi.Constr at_most_one_person_at_seat4>,
 <gurobi.Constr at_most_one_person_at_seat5>,
 <gurobi.Constr at_most_one_person_at_seat6>,
 <gurobi.Constr at_most_one_person_at_seat7>,
 <gurobi.Constr at_most_one_person_at_seat8>,
 <gurobi.Constr at_most_one_person_at_seat9>,
 <gurobi.Constr at_most_one_person_at_seat10>,
 <gurobi.Constr at_most_one_person_at_seat11>,
 <gurobi.Constr at_most_one_person_at_seat12>,
 <gurobi.Constr at_most_one_person_at_seat13>,
 <gurobi.Constr at_most_one_person_at_seat14>,
 <gurobi.Constr at_most_one_person_at_seat15>,
 <gurobi.Constr at_most_one_person_at_seat16>,
 <gurobi.Constr at_most_one_person_at_seat17>,
 <gurobi.Constr at_most_one_person_at_seat18>,
 <gurobi.Constr at_most_one_person_at_seat19>,
 <gurobi.Constr at_most_one_person_at_seat20>,
 <gurobi.Constr at_most

In [90]:
theater_seat_organizer.getObjective()

<gurobi.LinExpr: -15.0 + person0_inGroup0_atSeat0 + person0_inGroup0_atSeat1 + person0_inGroup0_atSeat2 + person0_inGroup0_atSeat3 + person0_inGroup0_atSeat4 + person0_inGroup0_atSeat5 + person0_inGroup0_atSeat6 + person0_inGroup0_atSeat7 + person0_inGroup0_atSeat8 + person0_inGroup0_atSeat9 + person0_inGroup0_atSeat10 + person0_inGroup0_atSeat11 + person0_inGroup0_atSeat12 + person0_inGroup0_atSeat13 + person0_inGroup0_atSeat14 + person0_inGroup0_atSeat15 + person0_inGroup0_atSeat16 + person0_inGroup0_atSeat17 + person0_inGroup0_atSeat18 + person0_inGroup0_atSeat19 + person0_inGroup0_atSeat20 + person0_inGroup0_atSeat21 + person0_inGroup0_atSeat22 + person0_inGroup0_atSeat23 + person0_inGroup0_atSeat24 + person0_inGroup0_atSeat25 + person0_inGroup0_atSeat26 + person0_inGroup0_atSeat27 + person0_inGroup0_atSeat28 + person0_inGroup0_atSeat29 + person0_inGroup0_atSeat30 + person0_inGroup0_atSeat31 + person0_inGroup0_atSeat32 + person0_inGroup0_atSeat33 + person0_inGroup0_atSeat34 + perso

In [91]:
theater_seat_organizer.getVars()

[<gurobi.Var person0_inGroup0_atSeat0 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat1 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat2 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat3 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat4 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat5 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat6 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat7 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat8 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat9 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat10 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat11 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat12 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat13 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat14 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat15 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat16 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat17 (value -0.0)>,
 <gurobi.Var person0_inGroup0_atSeat18